In [1]:
# coding: utf-8
import ast
import os
import sys
import re
from pprint import pprint
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


from func import blast 

run_name = 'test'

PATH = os.path.abspath(".")

if not os.path.isdir('%s/Data/%s'%(PATH,run_name)):
    os.makedirs('%s/Data/%s'%(PATH,run_name))
    os.makedirs('%s/Data/%s/fragmented_fasta'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast'%(PATH,run_name))
    os.makedirs('%s/Data/%s/barcode_counts'%(PATH,run_name))
    os.makedirs('%s/Data/%s/db'%(PATH,run_name))


In [6]:
###############################
## Create BLAST database
databse   = "%s/test_database.csv"%(PATH) #Change this to your own database
const_seq = "%s/const-seq.csv"%(PATH) # DO NOT CHANGE THIS

db_fna    = blast.mkdb(databse,const_seq,run_name,"%s/Data/%s/db"%(PATH,run_name))

command = "makeblastdb -in %s -out %s -dbtype nucl"%(db_fna,db_fna)

print(os.popen(command).read())

Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna


Building a new DB, current time: 07/26/2021 11:52:31
New DB name:   /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna
New DB title:  /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B



In [3]:
################################
## Convert fastq to fasta files in small chunks to make it easy to handle
miseq_dir = 'FULL/PATH/TO/FASTQ'
blast.fastq2fasta(miseq_dir,150,PATH)



FileNotFoundError: [Errno 2] No such file or directory: 'FULL/PATH/TO/FASTQ'